### Simple objective: branin with sleeps to emulate delay

In [1]:
# silence TF warnings and info messages, only print errors
# https://stackoverflow.com/questions/35911252/disable-tensorflow-debugging-information
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import numpy as np
import math
import timeit

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
def objective(points):
    if points.shape[1] != 2:
        raise ValueError(f"Incorrect input shape, expected (*, 2), got {x.shape}")

    def branin(x):
        x0 = x[..., :1] * 15.0 - 5.0
        x1 = x[..., 1:] * 15.0

        b = 5.1 / (4 * math.pi ** 2)
        c = 5 / math.pi
        r = 6
        s = 10
        t = 1 / (8 * math.pi)
        scale = 1
        translate = 10

        return scale * ((x1 - b * x0 ** 2 + c * x0 - r) ** 2 + s * (1 - t) * np.cos(x0) + translate)
        
    observations = []
    for point in points:
        observation = branin(point)
        observations.append(observation)
    
    return observations

In [4]:
objective(np.array([[0.1, 0.5]]))

[array([32.96369168])]

### Here comes Trieste

In [5]:
from trieste.objectives.utils import mk_observer
from trieste.space import Box

search_space = Box([0, 0], [1, 1])

In [6]:
from trieste.data import Dataset

num_initial_points = 3
initial_query_points = search_space.sample(num_initial_points)
initial_observations = objective(initial_query_points.numpy())

In [7]:
initial_data = Dataset(query_points=initial_query_points, observations=tf.constant(initial_observations, dtype=tf.float64))

print(initial_data)

Dataset(query_points=<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[0.05866792, 0.08743693],
       [0.65182824, 0.57646759],
       [0.97908318, 0.87241814]])>, observations=<tf.Tensor: shape=(3, 1), dtype=float64, numpy=
array([[185.22940936],
       [ 63.94383844],
       [108.50778776]])>)


In [8]:
import gpflow
from trieste.models import create_model
from trieste.utils import map_values
import tensorflow_probability as tfp

from trieste.models.gpflow.config import GPflowModelConfig


def build_model(data):
    variance = tf.math.reduce_variance(data.observations)
    kernel = gpflow.kernels.RBF(variance=variance)
    gpr = gpflow.models.GPR(data.astuple(), kernel, noise_variance=1e-5)
    gpflow.set_trainable(gpr.likelihood, False)

    return GPflowModelConfig(**{
        "model": gpr,
        "optimizer": gpflow.optimizers.Scipy(),
        "optimizer_args": {
            "minimize_args": {"options": dict(maxiter=100)},
        },
    })

In [9]:
model_spec = build_model(initial_data)
model = create_model(model_spec)

model.optimize(initial_data)

In [10]:
from trieste.acquisition import LocalPenalizationAcquisitionFunction
from trieste.acquisition.rule import EfficientGlobalOptimization

local_penalization_acq = LocalPenalizationAcquisitionFunction(search_space, num_samples=2000)
local_penalization_acq_rule = EfficientGlobalOptimization(
    num_query_points=2, builder=local_penalization_acq)
points_chosen = local_penalization_acq_rule.acquire_single(search_space, initial_data, model)

In [11]:
points_chosen

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[1., 0.],
       [1., 0.]])>

In [12]:
objective(points_chosen.numpy())

[array([10.96088904]), array([10.96088904])]

### Doing observations in separate processes

In [17]:
from multiprocessing import Process, Manager
import time
import psutil

num_workers = 2
num_observations = 10
dataset = initial_data

model_spec = build_model(initial_data)
model = create_model(model_spec)

model.optimize(initial_data)

local_penalization_acq = LocalPenalizationAcquisitionFunction(search_space)
local_penalization_acq_rule = EfficientGlobalOptimization(
    num_query_points=2, builder=local_penalization_acq)

m = Manager()
pq = m.Queue()
oq = m.Queue()

def observer_proc(points_queue, observations_queue, cpu_id):
    pid = os.getpid()
    
    current_process = psutil.Process()
    current_process.cpu_affinity([cpu_id])
    print(f"Process {pid}: set CPU to {cpu_id}", flush=True)
    
    while True:
        point_to_observe = points_queue.get()
        if (point_to_observe is None):
            return
        
        print(f"Process {pid}: observing data at point {point_to_observe}", flush=True)
        
        # insert some artificial delay
        # increases linearly with the absolute value of points
        # which means our evaluations will take different time, good for exploring async
        delay = 10 * np.sum(point_to_observe)
        print(f"Observer Process {pid} pretends like it's doing something for {delay:.2}s", flush=True)
        time.sleep(delay)
        new_observation = objective(point_to_observe)
        new_data = (point_to_observe, new_observation)
        
        print(f"Observer Process {pid}: observed data {new_data}", flush=True)
        
        observations_queue.put(new_data)


observer_processes = []

start = timeit.default_timer()
try:
    for i in range(psutil.cpu_count())[:num_workers]:
        observer_p = Process(target=observer_proc, args=(pq, oq, i))
        observer_p.daemon = True
        observer_p.start()

        observer_processes.append(observer_p)

    # init the queue with first batch of points
    points_chosen = local_penalization_acq_rule.acquire_single(search_space, initial_data, model)
    for point in points_chosen:
        pq.put(np.atleast_2d(point.numpy()))

    pending_points = points_chosen

    while len(dataset) < len(initial_data) + num_observations:
        pid = os.getpid()
        
        try:
            new_data = oq.get_nowait()
            print(f"Main Process {pid}: received data {new_data}", flush=True)
        except:
            continue
        
        query_points = tf.constant(new_data[0], dtype=tf.float64)

        # generate a tensor that contains True value for rows that equal to query_point
        # False otherwise
        # then use it to delete rows from pending points
        mask = tf.reduce_any(tf.not_equal(pending_points, query_points), axis=1)
        pending_points = tf.boolean_mask(pending_points, mask)
        print(pending_points)
        new_data = Dataset(query_points=query_points,
                           observations=tf.constant(new_data[1], dtype=tf.float64),
                          )
        dataset = dataset + new_data

        model.update(dataset)
        model.optimize(dataset)

        new_points = local_penalization_acq_rule.acquire_single(search_space, dataset, model, pending_points=pending_points)
        pending_points = tf.concat([pending_points, new_points], axis=0)
        new_points = new_points.numpy()
        print(f"Main Process {pid}: acquired point {new_points}", flush=True)
        for point in new_points:
            pq.put(np.atleast_2d(point))
finally:
    for prc in observer_processes:
        prc.terminate()
        prc.join()
        prc.close()
stop = timeit.default_timer()

print(f"Time : {stop - start}")

Process 31077: set CPU to 0
Process 31082: set CPU to 1
Process 31077: observing data at point [[0. 1.]]Process 31082: observing data at point [[1. 0.]]
Observer Process 31077 pretends like it's doing something for 1e+01s

Observer Process 31082 pretends like it's doing something for 1e+01s
Observer Process 31077: observed data (array([[0., 1.]]), [array([17.50829952])])
Main Process 24474: received data (array([[0., 1.]]), [array([17.50829952])])
Observer Process 31082: observed data (array([[1., 0.]]), [array([10.96088904])])
tf.Tensor([[1. 0.]], shape=(1, 2), dtype=float64)
Main Process 24474: acquired point [[1.         0.11334666]
 [1.         0.11334662]]
Process 31077: observing data at point [[1.         0.11334666]]Process 31082: observing data at point [[1.         0.11334662]]
Main Process 24474: received data (array([[1., 0.]]), [array([10.96088904])])

Observer Process 31077 pretends like it's doing something for 1.1e+01s
Observer Process 31082 pretends like it's doing som

In [32]:
dataset

Dataset(query_points=<tf.Tensor: shape=(7, 2), dtype=float64, numpy=
array([[0.52496236, 0.72011192],
       [0.99852951, 0.70053815],
       [0.39182544, 0.17070116],
       [0.        , 0.        ],
       [0.42677059, 0.19428767],
       [1.        , 0.        ],
       [1.        , 0.        ]])>, observations=<tf.Tensor: shape=(7, 1), dtype=float64, numpy=
array([[ 69.77988602],
       [ 58.48466555],
       [ 20.72779849],
       [308.12909601],
       [ 12.84672966],
       [ 10.96088904],
       [ 10.96088904]])>)